# Bulk-download WaPOR data 

In this notebook, we will use local Python module to automate the steps to read dataset metadata, request download url, download raster dataset, pre-process raster dataset using information from metadata (such as conversion factor, unit). *The datasets downloaded by this script will be automatically corrected by conversion factor*.

Run the code cell below to import packages and modules needed. You will be asked to provide WaPOR API token (see [Start here](0_Start_here.ipynb) page for how to get API token). The *WaPOR* module will remember your token and save it to the module folder. If you want to use a new API token, you have to delete this file **'modules\WaPOR\wapor_api_token.pickle'**

In [1]:
import os
import glob
import sys
import shapefile             
import matplotlib.pyplot as plt 

folder=r"..\..\modules"
sys.path.append(folder) #add folder with local modules to system paths
import WaPOR #import local module 'WaPOR'
WaPOR.API.getCatalog()

Loading WaPOR catalog...
Loading WaPOR catalog...Done


,code,caption,description,additionalInfo,tags,workspaceCode,dataType,index,operation,hidden,links,measure,dimension
0,L1_GBWP_A,Gross Biomass Water Productivity,The annual Gross Biomass Water Productivity ex...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 1, 'context': 'LEVELS...",WAPOR_2,RASTER,101,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
1,L1_NBWP_A,Net Biomass Water Productivity,The annual Net Biomass Water Productivity expr...,"{'format': 'Raster Dataset', 'unit': 'kg/m³ is...","[{'name': 'L1', 'index': 2, 'context': 'LEVELS...",WAPOR_2,RASTER,102,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WPR', 'caption': 'Water Productivity...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
2,L1_AETI_A,Actual EvapoTranspiration and Interception (An...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 3, 'context': 'LEVELS...",WAPOR_2,RASTER,103,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
3,L1_AETI_M,Actual EvapoTranspiration and Interception (Mo...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 4, 'context': 'LEVELS...",WAPOR_2,RASTER,104,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'MONTH', 'caption': 'Month', 'worksp..."
4,L1_AETI_D,Actual EvapoTranspiration and Interception (De...,The actual EvapoTranspiration and Interception...,"{'format': 'Raster Dataset', 'unit': 'mm', 'da...","[{'name': 'L1', 'index': 5, 'context': 'LEVELS...",WAPOR_2,RASTER,105,"{'pixelTimeSeries': True, 'areaStatsTimeSeries...",False,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'WATER_MM', 'caption': 'Amount of Wat...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,L2_BAS_PHE_S,"Phenology (Seasonal, clipped by basin)","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4151,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'PHE', 'caption': 'Phenology', 'descr...","[{'code': 'YEAR', 'caption': 'Year', 'descript..."
194,L2_BAS_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4161,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
195,L2_CTY_QUAL_NDVI_D,Quality of Normalized Difference Vegetation In...,"This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4162,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."
196,L2_BAS_QUAL_LST_D,"Quality Land Surface Temperature (Dekadal, cli...","This is a system dataset, used to let the down...",NaN,NaN,WAPOR_2,RASTER,4171,"{'pixelTimeSeries': False, 'areaStatsTimeSerie...",True,"[{'rel': 'self', 'href': 'https://io.apps.fao....","{'code': 'N_DAYS', 'caption': 'Number of Days'...","[{'code': 'DEKAD', 'caption': 'Dekad (10-Days ..."


## Dataset code

The **WaPOR** module contains the functions *download_dekadal*, *download_monthly*, and *download_yearly* that bulk-download WaPOR data for dekadal, monthly, and yearly temporal resolution. To sepecify the data layer to be downloaded, these functions require dataset code, bounding box coordiate, level of data, and starting and ending date of the period. The dataset code for each dataset is available in the table below:

| code        | Name          | 
| :---------: |:-------------:| 
| LCC | Land Cover Classification      | 
| PCP      | Precipitation      | 
| RET      | Reference Evapotranspiration      | 
| NPP      | Net Primary Production |
| AETI      | Actual Evapotranspiration and Interception |   
| E | Evaporation      |  
| I | Interception      | 
| T | Transpiration      | 
| PHE | Phenology      | 
| GBWP | Gross Biomass Water Productivity      | 

Use the selected data code instead of **RET** for the **data** parameter value in the code cells below.

## Example of WaPOR module functions

The example code cells below show how the *WaPOR* module was used to bulk-download WaPOR data: dekadal, monthly, and yearly Reference Evapotranspiration of an extent. The extent is defined by the bounding box of the shapefile of Awash basin. The raster data will be collected and pre-processed to the correct unit mm/period (mm/dekad, mm/month, or mm/year respectively), and save in the [output folder](data)


### Set download extent and output folder

Change xmin, ymin, xmax, ymax values to the download extent.
If needed, change *output_dir* path to the folder where you want to save data.

In [2]:
#define extent to download from a shapefile:
shape_fh=r".\data\Awash_shapefile.shp"
shape=shapefile.Reader(shape_fh)
xmin,ymin,xmax,ymax=shape.bbox

#define extent to download by coordinates:
# xmin,ymin,xmax,ymax=(32.65692516077674,-25.16412729789142,32.90420244933813,-25.01450956754679)

output_dir=r'.\data' # folder to save data

### Download daily data

Note that daily data is only available for Level 1 Reference Evapotranspiration and Precipitation

In [3]:
WaPOR.download_daily(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-01-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1,                    
                       )


Download WaPOR Level 1 dekadal RET data for the period 2009-01-01 till 2009-01-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_daily_L1_RET_E'

### Download dekadal data

**Note:** The unit of dekadal dataset will be converted to quantity/dekad instead of average daily quantity/day

In [4]:
WaPOR.download_dekadal(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 dekadal RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_dekadal_L1_RET_D'

### Download monthly data

In [5]:
WaPOR.download_monthly(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31', 
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 monthly RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_monthly_L1_RET_M'

### Download yearly data

In [6]:
WaPOR.download_yearly(output_dir, 
                       data='RET',
                       Startdate='2009-01-01', 
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 yearly RET data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_yearly_L1_RET_A'

### Download seasonal data

**Note**: Seasonal data is only available for some layers (e.g. Gross Biomass Water Productivity, Phenology)

In [7]:
WaPOR.download_seasonal(output_dir,
                       data='GBWP',
                       Startdate='2009-01-01',
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5],
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=2,
                       )
WaPOR.download_seasonal(output_dir,
                       data='PHE',
                       Startdate='2009-01-01',
                       Enddate='2009-12-31',
                       latlim=[ymin-0.5, ymax+0.5],
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=2,
                       )


Download WaPOR Level 2 seasonal GBWP data for the period 2009-01-01 till 2009-12-31
ERROR: Cannot get list of available data
ERROR: cannot get list of available data

Download WaPOR Level 2 seasonal PHE data for the period 2009-01-01 till 2009-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_seasonal_L2_PHE_S'

# Exercise

Download Level 1 monthly Precipitation, Actual Evapotranspiration and Interception, and yearly Land cover classification from 2009 to 2010 for the area bounded by the shapefile ".\data\Awash_shapefile.shp"

Use the **WaPOR** module functions following the given examples.

In [8]:
'''
Write your code here
'''

'\nWrite your code here\n'

In [3]:
WaPOR.download_monthly(output_dir, 
                       data='PCP',
                       Startdate='2009-01-01', 
                       Enddate='2010-12-31', 
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )

WaPOR.download_monthly(output_dir, 
                       data='AETI',
                       Startdate='2009-01-01', 
                       Enddate='2010-12-31', 
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )

WaPOR.download_yearly(output_dir, 
                       data='LCC',
                       Startdate='2009-01-01', 
                       Enddate='2010-12-31',
                       latlim=[ymin-0.5, ymax+0.5], 
                       lonlim=[xmin-0.5, xmax+0.5],
                       level=1, 
                       )


Download WaPOR Level 1 monthly PCP data for the period 2010-01-01 till 2010-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Download WaPOR Level 1 monthly AETI data for the period 2010-01-01 till 2010-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Download WaPOR Level 1 yearly LCC data for the period 2010-01-01 till 2010-12-31
Progress: |██████████████████████████████████████████████████| 100.0% Complete


'.\\data\\WAPOR.v2_yearly_L1_LCC_A'